## Retail Profitability & Discount Sensitivity Analysis

In [ ]:
import pandas as pd

df = pd.read_csv("SampleSuperstore.csv")
print(df.head())

In [ ]:
df.describe()

In [ ]:
print(df.dtypes)

### Creating Database and Schema
Ingestion of Data

In [ ]:
create or replace database retail_db;
create or replace schema retail_db.retail;
use schema retail_db.retail;

In [ ]:
create or replace file format csv_format
type = CSV
skip_header = 1
FIELD_OPTIONALLY_ENCLOSED_BY = '"';

In [ ]:
create or replace stage retail_stage
file_format = csv_format

In [ ]:
create or replace table sales_raw(
shipmode string, 
segment string, 
country string, 
city string, 
state string,
postalcode int, 
region string, 
category string, 
subcategory string, 
sales float, 
quantity int,
discount float,
profit float
)

In [ ]:
from snowflake.snowpark.context import get_active_session

# Get the current session
session = get_active_session()

session.file.put('SampleSuperstore.csv', '@retail_stage',auto_compress=False, overwrite=True)

In [ ]:
list @retail_stage

#### Copying stage to Table 

In [ ]:
COPY INTO sales_raw
FROM @retail_stage/SampleSuperstore.csv
FILE_FORMAT = csv_format;

In [ ]:
select * from sales_raw;

## Analytics

In [ ]:
desc table sales_raw;

In [ ]:
select count(*) from sales_raw;

In [ ]:
select count(*) from sales_raw 
WHERE shipmode IS NULL OR segment IS NULL OR country IS NULL OR 
      state IS NULL OR category IS NULL OR subcategory IS NULL OR
      city IS NULL OR postalcode IS NULL OR region IS NULL OR
      sales IS NULL OR quantity IS NULL OR discount IS NULL OR
      profit IS NULL;

#### How many countries, states and what are the shipment modes in the dataset?

In [ ]:
select distinct country from sales_raw;

In [ ]:
select count(distinct state) from sales_raw;

In [ ]:
select distinct shipmode from sales_raw

#### Do all countries follow all shipment modes?

In [ ]:
select count(distinct shipmode) as ship, state from sales_raw group by state
order by ship desc;

#### Which is the most profit turning state?

In [ ]:
select state, sum(profit) as profit_sum from sales_raw 
group by state
order by profit_sum desc;

#### Which region does the most sales?

In [ ]:
select sum(sales) as sale_sum, region 
from sales_raw
group by region
order by sale_sum desc;

#### Is west the most profitable as well?

In [ ]:
select region,
       sum(sales) s,
       sum(profit) p,
       round(sum(profit)/sum(sales)*100,2) profit_margin_pct
from sales_raw
group by region
order by p desc;

#### How many states do each region comprise?

In [ ]:
select count(state), region from sales_raw group by region order by count(state) desc;

In [ ]:
select sales, profit, discount, quantity from sales_raw where state='California'

In [ ]:
select min(sales), max(sales) from sales_raw;

#### Most profiting category and segment

In [ ]:
select sum(profit), category from sales_raw
group by category
order by sum(profit) desc;

In [ ]:
select category,
       sum(sales),
       sum(profit),
       round(sum(profit)/sum(sales)*100,2) margin_pct
from sales_raw
group by category
order by margin_pct desc;

In [ ]:
select sum(profit), segment from sales_raw
group by segment
order by sum(profit) desc;

In [ ]:
select segment, avg(discount)
from sales_raw
group by segment
order by avg(discount) desc;

Technology shows the highest profit and profit margin. Consumer segment is the most profitable and sells with more discount at average. 

#### where are we losing money?

In [ ]:
select sum(profit) as profit_sum, category, segment, region from sales_raw
group by category , segment, region
having profit_sum<0;

#### Does discount reduce profit?

In [ ]:
select discount, sum(profit) from sales_raw
group by discount
order by discount desc;

In [ ]:
select
  case
    when discount = 0 then '0'
    when discount <= 0.2 then '0–20%'
    when discount <= 0.4 then '20–40%'
    else '40%+'
  end as discount_bucket,
  sum(profit)
from sales_raw
group by 1
order by 1;

After offering discount of more than 20%, we start losing money

#### Worst performing sub-category

In [ ]:
select subcategory, sum(sales), sum(profit) from sales_raw
group by subcategory 
order by sum(profit) ;

In [ ]:
select subcategory,
sum(sales),
sum(profit),
round(sum(profit)/sum(sales)*100,2) margin_pct
from sales_raw
group by subcategory
order by margin_pct;

Point worth noting that even if the sales are good relatively, profits don't turn up for tables, bookcases, supplies.
Not gonna lie, it is heartbreaking to find bookcases in worst performing subcategory being a book lover myself. 

### Let's level up the insights
#### Profit margin by region-catgory

In [ ]:
select 
sum(sales) as s, 
sum(profit) as p,
round(sum(profit)/sum(sales)*100,2) as profit_margin,
region, category
from sales_raw
group by region, category
order by profit_margin desc;

#### Discount impact by subcategory

In [ ]:
select discount,
subcategory, sum(sales), sum(profit)
from sales_raw
group by subcategory , discount
order by subcategory, discount desc;

Irrespective of the subcategory once offered less than 20% or equal to 20% discount they start yielding profits. 

#### Category vs segment heatmap

In [ ]:
import seaborn as sn
import matplotlib.pyplot as pt


matrix = df.pivot_table(index="Category", columns="Segment", values="Profit")


pt.figure(figsize=(10, 6))
sn.heatmap(matrix, cmap="YlGnBu", annot=True, fmt=".1f")


pt.title("Profit with respect to category and segment")
pt.show()